In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# hyperparameters
n_epochs = 10
batch_size = 4
learning_rate = 0.003

In [3]:
# CIFAR-10 contains images with values in range [0, 1]
# we need to tormalize those to [-1, 1]
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [4]:
# dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True,
                                            transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True,
                                            transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size,
                                           shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# conv net implementation
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, len(classes))
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = ConvNet()

In [6]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [7]:
# training loop
n_total_steps = len(train_loader)

for epoch in range(n_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = [4, 3, 1024]
        # input_layer: 3 input channels, 6 output channels, kernel size 5x5
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 2000 == 0:
            print(f'epoch {epoch+1}/{n_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')
            
print('Finished training')

epoch 1/10, step 2000/12500, loss = 2.2843
epoch 1/10, step 4000/12500, loss = 2.0769
epoch 1/10, step 6000/12500, loss = 1.7509
epoch 1/10, step 8000/12500, loss = 1.7163
epoch 1/10, step 10000/12500, loss = 1.3268
epoch 1/10, step 12000/12500, loss = 1.2660
epoch 2/10, step 2000/12500, loss = 1.2117
epoch 2/10, step 4000/12500, loss = 1.6983
epoch 2/10, step 6000/12500, loss = 1.3690
epoch 2/10, step 8000/12500, loss = 1.5482
epoch 2/10, step 10000/12500, loss = 1.6349
epoch 2/10, step 12000/12500, loss = 0.8905
epoch 3/10, step 2000/12500, loss = 1.1690
epoch 3/10, step 4000/12500, loss = 2.8741
epoch 3/10, step 6000/12500, loss = 1.0510
epoch 3/10, step 8000/12500, loss = 1.0607
epoch 3/10, step 10000/12500, loss = 0.9592
epoch 3/10, step 12000/12500, loss = 1.6432
epoch 4/10, step 2000/12500, loss = 0.7308
epoch 4/10, step 4000/12500, loss = 1.2802
epoch 4/10, step 6000/12500, loss = 0.7746
epoch 4/10, step 8000/12500, loss = 1.1178
epoch 4/10, step 10000/12500, loss = 1.3667
epoc

In [8]:
# test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(len(classes))]
    n_class_samples = [0 for i in range(len(classes))]
    for images, labels in test_loader:
        outputs = model(images)
        # max returns (value, index)
        _, predictions = torch.max(outputs, dim=1)
        n_samples += len(labels)
        n_correct += (predictions == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predictions[i]
            if pred == label:
                n_class_correct[label] += 1
            n_class_samples[label] += 1
            
    acc = 100 * n_correct / n_samples
    print(f'Accuracy of the Network: {acc} %')
    
    for i in range(len(classes)):
        class_acc = 100 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of class {classes[i]}: {class_acc} %')

Accuracy of the Network: 63.47 %
Accuracy of class plane: 68.0 %
Accuracy of class car: 69.6 %
Accuracy of class bird: 47.8 %
Accuracy of class cat: 40.5 %
Accuracy of class deer: 67.7 %
Accuracy of class dog: 55.6 %
Accuracy of class frog: 61.3 %
Accuracy of class horse: 68.4 %
Accuracy of class ship: 79.2 %
Accuracy of class truck: 76.6 %
